In [6]:
import pandas as pd
import os

## Create Tables of agents' results

In [47]:
envs = ["FrozenLake", "CartPole"]
agents = ["Human", "Random", "HardCoded", "ML"]

In [14]:
def LoadStats(envName, runGroup, agent):
	path = os.path.join("Data", envName, runGroup, "replays", agent, "stats.tsv")
	df = pd.read_csv(path, sep="\t")
	return df

In [50]:
df = LoadStats("FrozenLake", "6", "ML")
df

,Terminated,Truncated,EpisodeTotalReward,EpisodeSteps,Agent,IsTrainingMode,NumEnvs,UseRealSim,MonteCarloConfig_MaxSecondsPerAction,MonteCarloConfig_CacheTreeBetweenActions,...,ModelConfigs_PlayStyleDiscriminator_TrainingBatchSize,ModelConfigs_PlayStyleDiscriminator_MaxDeploymentBatchSize,ModelConfigs_PlayStyleDiscriminator_DataTable,ModelConfigs_PlayStyleDiscriminator_LearningRate,ModelConfigs_PlayStyleDiscriminator_DenseLayers,ModelConfigs_PlayStyleDiscriminator_Activations,ModelConfigs_PlayStyleDiscriminator_Dropout,ModelConfigs_PlayStyleDiscriminator_L1,ModelConfigs_PlayStyleDiscriminator_L2,DataTables
0,False,True,0.0,11,ML,False,1,True,10,True,...,2048,2048,Human_Trajectories,0.001,"[256, 256]",relu,0.5,0.0,0.0,[{'TableName': 'Forward_NextState_Trajectories...
1,False,True,0.0,11,ML,False,1,True,10,True,...,2048,2048,Human_Trajectories,0.001,"[256, 256]",relu,0.5,0.0,0.0,[{'TableName': 'Forward_NextState_Trajectories...
2,False,True,0.0,11,ML,False,1,True,10,True,...,2048,2048,Human_Trajectories,0.001,"[256, 256]",relu,0.5,0.0,0.0,[{'TableName': 'Forward_NextState_Trajectories...
3,False,True,0.0,11,ML,False,1,True,10,True,...,2048,2048,Human_Trajectories,0.001,"[256, 256]",relu,0.5,0.0,0.0,[{'TableName': 'Forward_NextState_Trajectories...
4,False,True,0.0,11,ML,False,1,True,10,True,...,2048,2048,Human_Trajectories,0.001,"[256, 256]",relu,0.5,0.0,0.0,[{'TableName': 'Forward_NextState_Trajectories...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,True,False,1.0,6,ML,False,1,False,10,True,...,2048,2048,Human_Trajectories,0.001,"[256, 256]",relu,0.5,0.0,0.0,[{'TableName': 'Forward_NextState_Trajectories...
2685,True,False,1.0,6,ML,False,1,False,10,True,...,2048,2048,Human_Trajectories,0.001,"[256, 256]",relu,0.5,0.0,0.0,[{'TableName': 'Forward_NextState_Trajectories...
2686,True,False,1.0,6,ML,False,1,False,10,True,...,2048,2048,Human_Trajectories,0.001,"[256, 256]",relu,0.5,0.0,0.0,[{'TableName': 'Forward_NextState_Trajectories...
2687,True,False,1.0,6,ML,False,1,False,10,True,...,2048,2048,Human_Trajectories,0.001,"[256, 256]",relu,0.5,0.0,0.0,[{'TableName': 'Forward_NextState_Trajectories...


In [64]:
def EnvFitAgentSummary(summaries, df, agentName):
	mean = df["EpisodeTotalReward"].mean()
	sem = df["EpisodeTotalReward"].sem()

	summaries[f"{agentName} Mean"] = mean
	summaries[f"{agentName} sem"] = sem
	summaries[f"{agentName} n"] = len(df)

	return summaries

def GetEnvFitSummary(envName, runGroup):
	
	summaries = {}

	for agent in agents:
		try:
			df = LoadStats(envName, runGroup, agent)

			if agent == "ML":
				realSim = df.loc[df["UseRealSim"] == True]
				summaries = EnvFitAgentSummary(summaries, realSim, f"{agent}(Real Sim)")

				learnedSim = df.loc[df["UseRealSim"] == False]
				summaries = EnvFitAgentSummary(summaries, learnedSim, f"{agent}(Learned Model)")

			else:
				summaries = EnvFitAgentSummary(summaries, df, agent)
		except:
			pass

	# summaryDf = pd.DataFrame(summaries)
	return summaries

def GetEnvsSummary(runGroup):
	summaries = {}

	for env in envs:
		summaries[env] = GetEnvFitSummary(env, runGroup)

	df = pd.DataFrame(summaries)
	df = df.transpose()
	return df

df = GetEnvsSummary("6")
df


,Random Mean,Random sem,Random n,HardCoded Mean,HardCoded sem,HardCoded n,ML(Real Sim) Mean,ML(Real Sim) sem,ML(Real Sim) n,ML(Learned Model) Mean,ML(Learned Model) sem,ML(Learned Model) n
FrozenLake,0.004036,0.002015,991.0,1.0,0.0,990.0,0.719595,0.018478,592.0,1.0,0.0,2097.0
CartPole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,163.0,NaN,1.0


In [60]:
df.round(3)

,Random Mean,Random sem,Random n,HardCoded Mean,HardCoded sem,HardCoded n,ML(Real Sim) Mean,ML(Real Sim) sem,ML(Real Sim) n,ML(Learned Model) Mean,ML(Learned Model) sem,ML(Learned Model) n
FrozenLake,0.004,0.002,991.0,1.0,0.0,990.0,0.72,0.018,592.0,1.0,0.0,2097.0
CartPole,nan,nan,nan,nan,nan,nan,nan,nan,0.0,163.0,nan,1.0
